In [1]:
# read and convert MVO data to MiniSEED
import os
import glob
import obspy
MVOdatadir = '/data/Montserrat/LaharStudy/MVOdata'
mseeddir = os.path.join(MVOdatadir, 'miniseed')
SDS_DIR = '/data/SDS'
network = 'MV'

In [2]:
if not os.path.isdir(mseeddir):
    os.makedirs(mseeddir)
files = glob.glob(os.path.join(MVOdatadir, '*'))
for myfile in files:
    print(myfile)
    st = obspy.Stream()
    if myfile.endswith('.bz2'): 
        os.system(f'bunzip2 {myfile}')
        myfile = myfile.replace('.bz2', '')
        try:
            st = obspy.read(myfile, format='SEISAN')
        except:
            pass
    elif os.path.basename(myfile).startswith('2') or os.path.basename(myfile).startswith('M'):
        try:
            st = obspy.read(myfile)
        except:
            pass
    for tr in st:
        tr.stats.network = network
        if len(tr.stats.location)>0:
            tr.stats.channel = tr.stats.channel + tr.stats.location[0]
            if len(tr.stats.location)>1:
                tr.stats.location = tr.stats.location[1:]
            else:
                tr.stats.location = ''
        if tr.stats.sampling_rate == 1.0:
            tr.stats.channel = 'L' + tr.stats.channel[1:]
        if tr.stats.sampling_rate >= 80.0:
            tr.stats.channel = 'H' + tr.stats.channel[1:]
        if len(tr.stats.channel)==3:
            if tr.stats.channel[2]=='F':
                tr.stats.channel = tr.stats.channel[0] + 'D' + tr.stats.channel[2]
            else:
                tr.stats.channel = tr.stats.channel[0] + 'H' + tr.stats.channel[2]
            
                
        newfile = os.path.join(mseeddir, f'{tr.id}_{tr.stats.starttime}_{tr.stats.endtime}.mseed')
        if not os.path.isfile(newfile):
            try:
                tr.write(newfile, format='MSEED')
            except:
                print('could not write ',newfile)
        else:
            print(newfile,' already exists')
            

/data/Montserrat/LaharStudy/MVOdata/2018-10-15-2140-00S.MVO___046
/data/Montserrat/LaharStudy/MVOdata/miniseed/MV.OLV1..BHZ_2018-10-15T21:40:00.000000Z_2018-10-15T21:59:59.980000Z.mseed  already exists
/data/Montserrat/LaharStudy/MVOdata/miniseed/MV.OLV1..BHN_2018-10-15T21:40:00.000000Z_2018-10-15T21:59:59.980000Z.mseed  already exists
/data/Montserrat/LaharStudy/MVOdata/miniseed/MV.OLV1..BHE_2018-10-15T21:40:00.000000Z_2018-10-15T21:59:59.980000Z.mseed  already exists
/data/Montserrat/LaharStudy/MVOdata/2018-11-12-0000-00S.MVO___049
/data/Montserrat/LaharStudy/MVOdata/miniseed/MV.TRNT..LHZ_2018-11-12T00:00:00.000000Z_2018-11-12T00:19:59.000000Z.mseed  already exists
/data/Montserrat/LaharStudy/MVOdata/miniseed/MV.TRNT..LHN_2018-11-12T00:00:00.000000Z_2018-11-12T00:19:59.000000Z.mseed  already exists
/data/Montserrat/LaharStudy/MVOdata/miniseed/MV.TRNT..LHE_2018-11-12T00:00:00.000000Z_2018-11-12T00:19:59.000000Z.mseed  already exists
/data/Montserrat/LaharStudy/MVOdata/miniseed/MV.OLV2

In [4]:
# combine those MiniSEED files into day files, and organize them in SDS
# Input data extracted to this directory tree structure
dayst = obspy.Stream()
lastday = None
for filepath in sorted(glob.glob(os.path.join(mseeddir, '*.mseed'))):
    if os.path.isfile(filepath):
        try:
            st = obspy.read(filepath, format='MSEED')
        except Exception as e:
            print('\nUnknown format for ',filepath)
        else:
            startt = st[0].stats.starttime
            currentday = startt.day 
            if currentday != lastday:
                if len(dayst)>0:
                    for tr in dayst:
                        sdsfulldir = os.path.join(SDS_DIR, tr.stats.starttime.strftime('%Y'), tr.stats.network, tr.stats.station, tr.stats.channel+'.D')
                        sdsfullpath = os.path.join(sdsfulldir, f"{tr.id}.D.{tr.stats.starttime.strftime('%Y.%j')}")
                        if not os.path.isdir(sdsfulldir):
                            os.makedirs(sdsfulldir)
                        print(f'Writing {sdsfullpath}')
                        tr.write(sdsfullpath, format='MSEED')
                dayst = st.copy()
                for tr in dayst:
                    tr.stats.network = network
                lastday = currentday
            else:
                for tr in st:
                    tr.stats.network = network
                    dayst.append(tr.copy())
                dayst.merge(method=0, fill_value=0)

Writing /data/SDS/2018/MV/AIRS/BHE.D/MV.AIRS..BHE.D.2018.139
Writing /data/SDS/2018/MV/AIRS/BHN.D/MV.AIRS..BHN.D.2018.139
Writing /data/SDS/2018/MV/AIRS/BHZ.D/MV.AIRS..BHZ.D.2018.139
Writing /data/SDS/2018/MV/AIRS/LHE.D/MV.AIRS..LHE.D.2018.139
Writing /data/SDS/2018/MV/AIRS/LHN.D/MV.AIRS..LHN.D.2018.139
Writing /data/SDS/2018/MV/AIRS/LHZ.D/MV.AIRS..LHZ.D.2018.139
Writing /data/SDS/2018/MV/ANWB/BH1.D/MV.ANWB..BH1.D.2018.268
Writing /data/SDS/2018/MV/ANWB/BH1.D/MV.ANWB..BH1.D.2018.288
Writing /data/SDS/2018/MV/ANWB/BH1.D/MV.ANWB..BH1.D.2018.316
Writing /data/SDS/2019/MV/ANWB/BH1.D/MV.ANWB..BH1.D.2019.082
Writing /data/SDS/2018/MV/ANWB/BH2.D/MV.ANWB..BH2.D.2018.268
Writing /data/SDS/2018/MV/ANWB/BH2.D/MV.ANWB..BH2.D.2018.288
Writing /data/SDS/2018/MV/ANWB/BH2.D/MV.ANWB..BH2.D.2018.316
Writing /data/SDS/2019/MV/ANWB/BH2.D/MV.ANWB..BH2.D.2019.082
Writing /data/SDS/2018/MV/ANWB/BHZ.D/MV.ANWB..BHZ.D.2018.268
Writing /data/SDS/2018/MV/ANWB/BHZ.D/MV.ANWB..BHZ.D.2018.288
Writing /data/SDS/2018/M